# Assignment 2

# Tasks 1, 2, 3 of EXIST 2024

By Maha Usmani and Rania Siddiqui  


# Imports and setup


In [ ]:
!pip install datasets

In [ ]:

! pip install -U accelerate
! pip install -U transformers

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading the Training Dataset


In [ ]:
df = pd.read_json('/content/drive/MyDrive/EXIST/EXIST_2024_Dataset/EXIST 2024 Tweets Dataset/training/EXIST2024_training.json',orient='index')
df.head()

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,ethnicities_annotators,study_levels_annotators,countries_annotators,labels_task1,labels_task2,labels_task3,split
100001,100001,es,"@TheChiflis Ignora al otro, es un capullo.El p...",6,"[Annotator_1, Annotator_2, Annotator_3, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[White or Caucasian, Hispano or Latino, White ...","[Bachelor’s degree, Bachelor’s degree, High sc...","[Italy, Mexico, United States, Spain, Spain, C...","[YES, YES, NO, YES, YES, YES]","[REPORTED, JUDGEMENTAL, -, REPORTED, JUDGEMENT...","[[OBJECTIFICATION], [OBJECTIFICATION, SEXUAL-V...",TRAIN_ES
100002,100002,es,@ultimonomada_ Si comicsgate se parece en algo...,6,"[Annotator_7, Annotator_8, Annotator_9, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[Black or African American, Hispano or Latino,...","[High school degree or equivalent, Bachelor’s ...","[United Kingdom, Mexico, United States, Portug...","[NO, NO, NO, NO, YES, NO]","[-, -, -, -, DIRECT, -]","[[-], [-], [-], [-], [OBJECTIFICATION], [-]]",TRAIN_ES
100003,100003,es,"@Steven2897 Lee sobre Gamergate, y como eso ha...",6,"[Annotator_7, Annotator_8, Annotator_9, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[Black or African American, Hispano or Latino,...","[High school degree or equivalent, Bachelor’s ...","[United Kingdom, Mexico, United States, Portug...","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",TRAIN_ES
100004,100004,es,@Lunariita7 Un retraso social bastante lamenta...,6,"[Annotator_13, Annotator_14, Annotator_15, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[Hispano or Latino, Hispano or Latino, White o...","[High school degree or equivalent, Bachelor’s ...","[Mexico, Chile, Spain, Spain, Portugal, Spain]","[NO, NO, YES, NO, YES, YES]","[-, -, DIRECT, -, REPORTED, REPORTED]","[[-], [-], [IDEOLOGICAL-INEQUALITY], [-], [IDE...",TRAIN_ES
100005,100005,es,@novadragon21 @icep4ck @TvDannyZ Entonces como...,6,"[Annotator_19, Annotator_20, Annotator_21, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[Hispano or Latino, Hispano or Latino, White o...","[Bachelor’s degree, Bachelor’s degree, Master’...","[Mexico, Afghanistan, United States, Italy, Po...","[YES, NO, YES, NO, YES, YES]","[REPORTED, -, JUDGEMENTAL, -, JUDGEMENTAL, DIR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [-...",TRAIN_ES


In [ ]:
required_columns = ["id_EXIST", "lang", "tweet","labels_task1","labels_task2","labels_task3","split"]
df = df[required_columns]

## EDA

In [ ]:
un = df["annotators"].apply(str).unique()
un

In [ ]:
df["annotators"] = df['annotators'].apply(str)

In [ ]:
annotator_counts = df[["annotators","id_EXIST"]].groupby("annotators").count()
annotator_counts

,id_EXIST
annotators,
"['Annotator_1', 'Annotator_2', 'Annotator_3', 'Annotator_4', 'Annotator_5', 'Annotator_6']",57
"['Annotator_103', 'Annotator_104', 'Annotator_105', 'Annotator_106', 'Annotator_107', 'Annotator_108']",57
"['Annotator_109', 'Annotator_110', 'Annotator_111', 'Annotator_112', 'Annotator_113', 'Annotator_114']",57
"['Annotator_115', 'Annotator_116', 'Annotator_117', 'Annotator_118', 'Annotator_119', 'Annotator_120']",57
"['Annotator_121', 'Annotator_122', 'Annotator_123', 'Annotator_124', 'Annotator_125', 'Annotator_126']",57
...,...
"['Annotator_73', 'Annotator_74', 'Annotator_75', 'Annotator_76', 'Annotator_77', 'Annotator_78']",57
"['Annotator_79', 'Annotator_80', 'Annotator_81', 'Annotator_82', 'Annotator_83', 'Annotator_84']",57
"['Annotator_85', 'Annotator_86', 'Annotator_87', 'Annotator_88', 'Annotator_89', 'Annotator_90']",57


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


# Loading the Validation Dataset

In [ ]:
df_val = pd.read_json('/content/drive/MyDrive/EXIST/EXIST_2024_Dataset/EXIST 2024 Tweets Dataset/dev/EXIST2024_dev.json',orient='index')


In [ ]:
df_val = df_val[required_columns]

In [ ]:
df_val

,id_EXIST,lang,tweet,labels_task1,labels_task2,labels_task3,split,hard_label1,hard_label2,hard_label3,soft_label1,soft_label2,soft_label3
300001,300001,es,@Fichinescu La comunidad gamer es un antro de ...,"[NO, YES, YES, NO, YES, NO]","[-, JUDGEMENTAL, JUDGEMENTAL, -, REPORTED, -]","[[-], [MISOGYNY-NON-SEXUAL-VIOLENCE], [MISOGYN...",DEV_ES,0,-,"[-, MISOGYNY-NON-SEXUAL-VIOLENCE]",NaN,NaN,NaN
300002,300002,es,@anacaotica88 @MordorLivin No me acuerdo de lo...,"[YES, YES, NO, YES, YES, YES]","[JUDGEMENTAL, REPORTED, -, JUDGEMENTAL, JUDGEM...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",DEV_ES,1,JUDGEMENTAL,"[MISOGYNY-NON-SEXUAL-VIOLENCE, IDEOLOGICAL-INE...",NaN,NaN,NaN
300003,300003,es,@cosmicJunkBot lo digo cada pocos dias y lo re...,"[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",DEV_ES,0,-,-,NaN,NaN,NaN
300004,300004,es,Also mientras les decia eso la señalaba y deci...,"[NO, YES, YES, YES, YES, YES]","[-, REPORTED, REPORTED, REPORTED, JUDGEMENTAL,...","[[-], [SEXUAL-VIOLENCE], [SEXUAL-VIOLENCE], [S...",DEV_ES,1,REPORTED,SEXUAL-VIOLENCE,NaN,NaN,NaN
300005,300005,es,"And all people killed, attacked, harassed by ...","[NO, YES, NO, NO, NO, NO]","[-, DIRECT, -, -, -, -]","[[-], [STEREOTYPING-DOMINANCE], [-], [-], [-],...",DEV_ES,0,-,-,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
400485,400485,en,@YesReallyAngel “Don’t wear a black bra with a...,"[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, JUDGEMENTAL, DIRECT, DIRECT, ...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, SEX...",DEV_EN,1,DIRECT,"[OBJECTIFICATION, STEREOTYPING-DOMINANCE, SEXU...",NaN,NaN,NaN
400486,400486,en,""" get changed , you look like a prostitute . ""...","[YES, YES, YES, YES, NO, YES]","[DIRECT, REPORTED, REPORTED, REPORTED, -, DIRECT]","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",DEV_EN,1,REPORTED,"[SEXUAL-VIOLENCE, OBJECTIFICATION, IDEOLOGICAL...",NaN,NaN,NaN
400487,400487,en,made this top and my mom gave me the “you look...,"[YES, YES, YES, YES, YES, NO]","[JUDGEMENTAL, REPORTED, REPORTED, REPORTED, RE...","[[OBJECTIFICATION], [OBJECTIFICATION], [OBJECT...",DEV_EN,1,REPORTED,OBJECTIFICATION,NaN,NaN,NaN
400488,400488,en,@DawnAnd91320913 I haven't seen anything that ...,"[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, DIRECT]","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",DEV_EN,1,DIRECT,"[STEREOTYPING-DOMINANCE, IDEOLOGICAL-INEQUALIT...",NaN,NaN,NaN


# Preproccessing the Dataset


In [ ]:
def assign_majority_label(labels, num_annotators):
    if num_annotators == 1:labels = np.concatenate(labels)
    else:labels = np.array(labels).flatten()
    counts = pd.Series(labels).value_counts()
    filtered_counts = counts[counts > num_annotators]
    if len(filtered_counts.index) == 1:
        return filtered_counts.index[0]
    elif len(filtered_counts.index) > 1:
        return filtered_counts.index.tolist()
    else:
        return None

In [ ]:
def transform(x):
    if x == "YES":return 1
    return 0


In [ ]:
def transform2(x):
    if x == "DIRECT":return 0
    elif x == "JUDGEMENTAL":return 1
    elif x == "REPORTED":return 2
    else:return 3

In [ ]:
def transform3(x):
    out = []
    if x:
      for i in x:
        if i=='IDEOLOGICAL-INEQUALITY':
          out.append(0)
        elif i=='MISOGYNY-NON-SEXUAL-VIOLENCE':
          out.append(1)
        elif i=='OBJECTIFICATION':
          out.append(2)
        elif i=='STEREOTYPING-DOMINANCE':
          out.append(3)
        elif i=='SEXUAL-VIOLENCE':
          out.append(4)
        elif i=="-":
          out.append(5)
    return out

In [ ]:
labels1 = ["YES","NO"]
def calculate_soft_labels_1(label):
    d = {label:0 for label in labels1}
    for i in label:
        d[i] = d.get(i, 0) + 1
    return softmax(d)

labels2 = ["DIRECT","JUDGEMENTAL","REPORTED","NO"]
def calculate_soft_labels_2(label):
    d = {label:0 for label in labels2}
    for i in label:
        if i=='-' or i=="UNKNOWN":i = "NO"
        d[i] = d.get(i, 0) + 1
    return softmax(d)

In [ ]:
labels3 = ['IDEOLOGICAL-INEQUALITY','MISOGYNY-NON-SEXUAL-VIOLENCE', 'OBJECTIFICATION','STEREOTYPING-DOMINANCE','SEXUAL-VIOLENCE','NO']
def softmax(label_counts):
    labels = list(label_counts.values())
    probabilities = np.exp(labels)/np.sum(np.exp(labels))
    return probabilities
def calculate_soft_labels_3(labels):
    d = {label: 0 for label in labels3}
    for label in labels:
        for j in label:
            if j =='-' or j =="UNKNOWN":j = 'NO'
            d[j] = d.get(j,0) + 1
    return softmax(d)

In [ ]:
df["hard_label1"] = df["labels_task1"].apply(lambda x: assign_majority_label(x, 3))
df["hard_label1"] = df["hard_label1"].apply(transform)
df["hard_label2"] = df["labels_task2"].apply(lambda x: assign_majority_label(x, 2))
df["hard_label2"] = df["hard_label2"].apply(transform2)
df["hard_label3"] = df["labels_task3"].apply(lambda x: assign_majority_label(x, 1))
df["hard_label3"] = df["hard_label3"].apply(transform3)

df["soft_label1"] = df["labels_task1"].apply(calculate_soft_labels_1)
df["soft_label2"] = df["labels_task2"].apply(calculate_soft_labels_2)
df["soft_label3"] = df["labels_task3"].apply(calculate_soft_labels_3)

In [ ]:
df_val["hard_label1"] = df_val["labels_task1"].apply(lambda x: assign_majority_label(x, 3))
df_val["hard_label1"] = df_val["hard_label1"].apply(transform)
df_val["hard_label2"] = df_val["labels_task2"].apply(lambda x: assign_majority_label(x, 2))
df_val["hard_label2"] = df_val["hard_label2"].apply(transform2)
df_val["hard_label3"] = df_val["labels_task3"].apply(lambda x: assign_majority_label(x, 1))
df_val["hard_label3"] = df_val["hard_label3"].apply(transform3)

df_val["soft_label1"] = df_val["labels_task1"].apply(calculate_soft_labels_1)
df_val["soft_label2"] = df_val["labels_task2"].apply(calculate_soft_labels_2)
df_val["soft_label3"] = df_val["labels_task3"].apply(calculate_soft_labels_3)

In [ ]:
train_dataset_hard1 = df[["tweet", "hard_label1"]]
train_dataset_hard2 = df[["tweet", "hard_label2"]]
train_dataset_hard3 = df[["tweet", "hard_label3"]]

train_dataset_soft1 = df[["tweet", "soft_label1"]]
train_dataset_soft2 = df[["tweet", "soft_label2"]]
train_dataset_soft3 = df[["tweet", "soft_label3"]]

val_dataset_hard1 = df_val[["tweet", "hard_label1"]]
val_dataset_hard2 = df_val[["tweet", "hard_label2"]]
val_dataset_hard3 = df_val[["tweet", "hard_label3"]]

val_dataset_soft1 = df_val[["tweet", "soft_label1"]]
val_dataset_soft2 = df_val[["tweet", "soft_label2"]]
val_dataset_soft3 = df_val[["tweet", "soft_label3"]]

In [ ]:
def equalize_vectors(x):
    out = [0] * 6
    for i in x:
      out[i] = 1
    return out


In [ ]:
train_dataset_hard3["hard_label3"] = train_dataset_hard3["hard_label3"].apply(equalize_vectors)
val_dataset_hard3["hard_label3"] = val_dataset_hard3["hard_label3"].apply(equalize_vectors)

<ipython-input-21-748930d3a41c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset_hard3["hard_label3"] = train_dataset_hard3["hard_label3"].apply(equalize_vectors)
<ipython-input-21-748930d3a41c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_dataset_hard3["hard_label3"] = val_dataset_hard3["hard_label3"].apply(equalize_vectors)


# Model Loading and Tokenizer

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
model_name = "bert-base-multilingual-uncased"


def tokenize_function(example):
    return tokenizer(
        example["tweet"],
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )

# Task 1

## Model loading

In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels = 2)
tokenizer = BertTokenizer.from_pretrained(model_name)

## Tokenization

In [ ]:
tokenized_train = Dataset.from_pandas(train_dataset_hard1).map(tokenize_function, batched=True)
tokenized_validation = Dataset.from_pandas(val_dataset_hard1).map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets = tokenized_train.rename_column("hard_label1", "labels")
tokenized_datasets_train = tokenized_datasets.remove_columns(["tweet", "__index_level_0__"])
tokenized_datasets_train.set_format("torch")


tokenized_datasets = tokenized_validation.rename_column("hard_label1", "labels")
tokenized_datasets_val = tokenized_datasets.remove_columns(["tweet", "__index_level_0__"])
tokenized_datasets_val .set_format("torch")

## Training the dataset

In [ ]:
from transformers import Trainer
import torch
torch.cuda.empty_cache()

training_args = TrainingArguments(
    output_dir="./content/drive/MyDrive/EXIST_2024/output",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    learning_rate=2e-05,
    weight_decay=0.004891290652279793,
    logging_dir="./content/drive/MyDrive/EXIST_2024/logs",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch"
)


torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

trainer = Trainer( # we only trained the model till 1 epochs since it was taking too much time to train for 3.
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset= tokenized_datasets_val
)

trainer.train()

## Predictions and Evaluations for Task 1

In [ ]:
predictions, labels, metrics = trainer.predict(tokenized_datasets_val, metric_key_prefix="predict")

Epoch,Training Loss,Validation Loss
1,0.585600,0.575337


In [ ]:
predictions_sigmoid = torch.sigmoid(torch.from_numpy(predictions))
output = torch.where(predictions_sigmoid[:, 0] > 0.5, 1, 0)

In [ ]:
val_dataset_hard1["prediction"] = output

In [ ]:
MSE = np.mean(np.array((output - labels))**2)
print("Mean Squared Error (MSE):", MSE)

Mean Squared Error (MSE): 0.7658959537572254


# Task 2


## Loading the model and tokenizer

In [ ]:
model_name = "bert-base-multilingual-uncased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels = 4)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Tokenizing the dataset

In [ ]:
from datasets import Dataset

In [ ]:
tokenized_train2 = Dataset.from_pandas(train_dataset_hard2).map(tokenize_function, batched=True)
tokenized_validation2 = Dataset.from_pandas(val_dataset_hard2).map(tokenize_function, batched=True)

Map:   0%|          | 0/1038 [00:00<?, ? examples/s]

{'tweet': ['@Fichinescu La comunidad gamer es un antro de misóginos y supremacistas blancos hace años pero hoy día pocos se acuerdan de gamergate y todo lo que causó, y en cierto sentido sigue causando.', '@anacaotica88 @MordorLivin No me acuerdo de los detalles de GamerGate, pero ella estuvo en el ojo del huracán recibiendo acoso de hombres indignados (y sus medios frikis) y creo que también acosaron a Brianna Wu, q es transfemenino. Seguramente tuvo eso que ver y quiso "cerrar filas".', '@cosmicJunkBot lo digo cada pocos dias y lo repito: TODO esto se pudo evitar si se hubiera parado el gamergate a tiempo', 'Also mientras les decia eso la señalaba y decia que como el era del staff, la iban a ban de ese evento. Intel behavior much? Este cabron has sexually harassed women and girls en el community y cuando es confrontado, lo negative y mienta. Ahora salio con esta, HE should be banned.', 'And all people killed,  attacked, harassed by CAM, Mapuche Resistance? Siches is a real,piece of w

In [ ]:
tokenized_datasets2 = tokenized_train2.rename_column("hard_label2", "labels")
tokenized_datasets_train2 = tokenized_datasets2.remove_columns(["tweet", "__index_level_0__"])
tokenized_datasets_train2.set_format("torch")


tokenized_datasets2 = tokenized_validation2.rename_column("hard_label2", "labels")
tokenized_datasets_val2 = tokenized_datasets2.remove_columns(["tweet", "__index_level_0__"])
tokenized_datasets_val2.set_format("torch")

## Training

In [ ]:
# torch.cuda.empty_cache()

training_args = TrainingArguments(
    output_dir="./content/drive/MyDrive/EXIST_2024/output2",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    learning_rate=2e-05,
    weight_decay=0.004891290652279793,
    logging_dir="./content/drive/MyDrive/EXIST_2024/logs2",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch"
)


torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train2,
    eval_dataset= tokenized_datasets_val2
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.806900,0.809707


TrainOutput(global_step=1730, training_loss=0.806861127732117, metrics={'train_runtime': 804.7039, 'train_samples_per_second': 8.599, 'train_steps_per_second': 2.15, 'total_flos': 1820761198264320.0, 'train_loss': 0.806861127732117, 'epoch': 1.0})

## Predictions and Evaluations for Task 2

In [ ]:
predictions, labels, metrics = trainer.predict(tokenized_datasets_val2, metric_key_prefix="predict")

In [ ]:
predictions_sigmoid = torch.sigmoid(torch.from_numpy(predictions))

In [ ]:
print(predictions_sigmoid)
labels2 = ["DIRECT","JUDGEMENTAL","REPORTED","NO"]
out = []
for i in predictions_sigmoid:
    out.append(labels2[torch.argmax(i)])

In [ ]:
val_dataset_hard2["prediction"] = out

<ipython-input-62-d02c601dfc04>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_dataset_hard2["prediction"] = out


In [ ]:
val_dataset_hard2

,tweet,hard_label2,prediction
300001,@Fichinescu La comunidad gamer es un antro de ...,3,NO
300002,@anacaotica88 @MordorLivin No me acuerdo de lo...,1,NO
300003,@cosmicJunkBot lo digo cada pocos dias y lo re...,3,NO
300004,Also mientras les decia eso la señalaba y deci...,2,NO
300005,"And all people killed, attacked, harassed by ...",3,NO
...,...,...,...
400485,@YesReallyAngel “Don’t wear a black bra with a...,0,DIRECT
400486,""" get changed , you look like a prostitute . ""...",2,DIRECT
400487,made this top and my mom gave me the “you look...,2,NO
400488,@DawnAnd91320913 I haven't seen anything that ...,0,NO


# Task3

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset

## Loading the model and tokenizer

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
model_name = "bert-base-multilingual-uncased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels = 6)
tokenizer = BertTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [ ]:
train_dataset_hard3.head()

,tweet,hard_label3
100001,"@TheChiflis Ignora al otro, es un capullo.El p...","[0, 0, 1, 0, 1, 0]"
100002,@ultimonomada_ Si comicsgate se parece en algo...,"[0, 0, 0, 0, 0, 1]"
100003,"@Steven2897 Lee sobre Gamergate, y como eso ha...","[0, 0, 0, 0, 0, 1]"
100004,@Lunariita7 Un retraso social bastante lamenta...,"[1, 0, 0, 0, 0, 1]"
100005,@novadragon21 @icep4ck @TvDannyZ Entonces como...,"[1, 0, 1, 1, 0, 1]"


In [ ]:
train_dataset_hard3 = train_dataset_hard3[train_dataset_hard3.hard_label3.apply(lambda x: x != [])]
val_dataset_hard3 = val_dataset_hard3[val_dataset_hard3.hard_label3.apply(lambda x: x != [])]

## Tokenizing the Dataset

In [ ]:
tokenized_train3 = Dataset.from_pandas(train_dataset_hard3).map(tokenize_function, batched = True)
tokenized_validation3 = Dataset.from_pandas(val_dataset_hard3).map(tokenize_function, batched = True)

Map:   0%|          | 0/6920 [00:00<?, ? examples/s]

{'tweet': ['@TheChiflis Ignora al otro, es un capullo.El problema con este youtuber denuncia el acoso... cuando no afecta a la gente de izquierdas. Por ejemplo, en su video sobre el gamergate presenta como "normal" el acoso que reciben Fisher, Anita o Zöey cuando hubo hasta amenazas de bomba.', '@ultimonomada_ Si comicsgate se parece en algo a gamergate pues muy bien por el acoso. Y si se está haciendo un sabotaje porque hay personajes que no os gustan entonces gracias por darme la razón. Sois unos lloricas ofendidos.', '@Steven2897 Lee sobre Gamergate, y como eso ha cambiado la manera en la cual nos comunicamos en el internet. Los fanboys de Halo están tóxicos pero los fanboys de otras comunidades/juegos también han querido coger pauta con eso 🤷🏾\u200d♂️', '@Lunariita7 Un retraso social bastante lamentable, gamergate, trump, y aquí en españa pues lo que tenemos...', '@novadragon21 @icep4ck @TvDannyZ Entonces como así es el mercado lo mejor no es hacer algo para cambiarlo y seguir alim

Map:   0%|          | 0/1038 [00:00<?, ? examples/s]

{'tweet': ['@Fichinescu La comunidad gamer es un antro de misóginos y supremacistas blancos hace años pero hoy día pocos se acuerdan de gamergate y todo lo que causó, y en cierto sentido sigue causando.', '@anacaotica88 @MordorLivin No me acuerdo de los detalles de GamerGate, pero ella estuvo en el ojo del huracán recibiendo acoso de hombres indignados (y sus medios frikis) y creo que también acosaron a Brianna Wu, q es transfemenino. Seguramente tuvo eso que ver y quiso "cerrar filas".', '@cosmicJunkBot lo digo cada pocos dias y lo repito: TODO esto se pudo evitar si se hubiera parado el gamergate a tiempo', 'Also mientras les decia eso la señalaba y decia que como el era del staff, la iban a ban de ese evento. Intel behavior much? Este cabron has sexually harassed women and girls en el community y cuando es confrontado, lo negative y mienta. Ahora salio con esta, HE should be banned.', 'And all people killed,  attacked, harassed by CAM, Mapuche Resistance? Siches is a real,piece of w

In [ ]:
tokenized_datasets3 = tokenized_train3.rename_column("hard_label3", "labels")
tokenized_datasets_train3 = tokenized_datasets3.remove_columns(["tweet", "__index_level_0__","token_type_ids","attention_mask"])
tokenized_datasets_train3.set_format("torch")


tokenized_datasets3 = tokenized_validation3.rename_column("hard_label3", "labels")
tokenized_datasets_val3 = tokenized_datasets3.remove_columns(["tweet", "__index_level_0__","token_type_ids","attention_mask"])
tokenized_datasets_val3.set_format("torch")

In [ ]:

tokenized_datasets_train3

Dataset({
    features: ['labels', 'input_ids'],
    num_rows: 6920
})

## Training the model

In [ ]:
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss()
        loss = loss_fct(logits.view(-1, 6),
                        labels.float().view(-1, 6))
        return (loss, outputs) if return_outputs else loss

trainer = MultilabelTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets_train3[:500],
    eval_dataset = tokenized_datasets_val3[:200],
    tokenizer=tokenizer
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.461500,0.476224


TrainOutput(global_step=1730, training_loss=0.4615043155030708, metrics={'train_runtime': 866.1654, 'train_samples_per_second': 7.989, 'train_steps_per_second': 1.997, 'total_flos': 1820793893437440.0, 'train_loss': 0.4615043155030708, 'epoch': 1.0})

In [ ]:
pip install numpy --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 28.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2


## Predictions

In [ ]:
predictions, labels, metrics = trainer.predict(tokenized_datasets_val3, metric_key_prefix="predict")

In [ ]:
predictions_sigmoid = torch.sigmoid(torch.from_numpy(predictions))
output = torch.where(predictions_sigmoid[:, 0] > 0.5, 1, 0)

In [ ]:
result = (np.array(predictions_sigmoid)>0.4).astype(int)

In [ ]:
print(predictions_sigmoid)
out = []
for i in predictions_sigmoid:
    out.append(labels3[torch.argmax(i)])

tensor([[0.1121, 0.0904, 0.1276, 0.1432, 0.0795, 0.8924],
        [0.2975, 0.1305, 0.2255, 0.3518, 0.1101, 0.7591],
        [0.1050, 0.0958, 0.1309, 0.1357, 0.0839, 0.8955],
        ...,
        [0.1136, 0.0925, 0.1330, 0.1473, 0.0804, 0.8888],
        [0.3247, 0.1364, 0.2355, 0.3799, 0.1133, 0.7209],
        [0.1087, 0.0944, 0.1331, 0.1386, 0.0827, 0.8934]])


In [ ]:
out

In [ ]:
val_dataset_hard3["prediction"] = out

<ipython-input-45-e26802c1cf98>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_dataset_hard3["prediction"] = out


In [ ]:
val_dataset_hard3

,tweet,hard_label3,prediction
300001,@Fichinescu La comunidad gamer es un antro de ...,"[0, 1, 0, 0, 0, 1]",NO
300002,@anacaotica88 @MordorLivin No me acuerdo de lo...,"[1, 1, 0, 1, 0, 0]",NO
300003,@cosmicJunkBot lo digo cada pocos dias y lo re...,"[0, 0, 0, 0, 0, 1]",NO
300004,Also mientras les decia eso la señalaba y deci...,"[0, 0, 0, 0, 0, 1]",NO
300005,"And all people killed, attacked, harassed by ...","[0, 0, 0, 0, 0, 1]",NO
...,...,...,...
400485,@YesReallyAngel “Don’t wear a black bra with a...,"[0, 1, 1, 1, 1, 0]",NO
400486,""" get changed , you look like a prostitute . ""...","[1, 0, 1, 0, 1, 0]",NO
400487,made this top and my mom gave me the “you look...,"[0, 0, 0, 0, 0, 0]",NO
400488,@DawnAnd91320913 I haven't seen anything that ...,"[1, 1, 1, 1, 0, 0]",NO
